In [1]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import os

# Charger les données
df = pd.read_parquet("data/processed/games_raw.parquet")

# Filtrage des ouvertures inutiles
df = df[df["main_opening"].notna() &
        (df["main_opening"] != "?") &
        (df["main_opening"] != "King's Pawn") &
        (df["main_opening"] != "Queen's Pawn")]

# Blancs
white_openings = df.groupby("main_opening").agg(
    games_white=("white_score", "count"),
    winrate_white=("white_score", "mean")
).reset_index()
white_openings = white_openings[white_openings["games_white"] >= 30]
top_white_winrate = white_openings.sort_values("winrate_white", ascending=False).head(10)
top_white_games = white_openings.sort_values("games_white", ascending=False).head(10)

# Noirs
black_openings = df.groupby("main_opening").agg(
    games_black=("black_score", "count"),
    winrate_black=("black_score", "mean")
).reset_index()
black_openings = black_openings[black_openings["games_black"] >= 30]
top_black_winrate = black_openings.sort_values("winrate_black", ascending=False).head(10)
top_black_games = black_openings.sort_values("games_black", ascending=False).head(10)

def create_figure(top_winrate, top_games, color_winrate, color_games, side_icon, side_name):
    fig = go.Figure()

    # Winrate trace
    fig.add_trace(go.Bar(
        x=top_winrate["main_opening"],
        y=top_winrate[f"winrate_{side_name}"],
        customdata=top_winrate[[f"games_{side_name}"]],
        marker=dict(
            color=top_winrate[f"winrate_{side_name}"],
            colorscale=color_winrate,
            line=dict(color='black', width=1.5),
            showscale=False
        ),
        text=top_winrate[f"winrate_{side_name}"].apply(lambda x: f"{x:.2%}"),
        textposition="outside",
        name="Trié par Winrate",
        hovertemplate=f"<b>%{{x}}</b><br>Winrate: %{{y:.2%}}<br>Parties: %{{customdata[0]}}<extra></extra>"
    ))

    # Games trace
    fig.add_trace(go.Bar(
        x=top_games["main_opening"],
        y=top_games[f"games_{side_name}"],
        marker=dict(
            color=top_games[f"games_{side_name}"],
            colorscale=color_games,
            line=dict(color='black', width=1.5),
            showscale=False
        ),
        text=top_games[f"games_{side_name}"],
        textposition="outside",
        name="Trié par Nombre de parties",
        hovertemplate=f"<b>%{{x}}</b><br>Parties: %{{y}}<extra></extra>",
        visible=False  # Par défaut, masqué
    ))

    # Annotation pour le meilleur winrate
    best = top_winrate.iloc[0]
    annotation = dict(
        x=best["main_opening"],
        y=best[f"winrate_{side_name}"],
        text=f"⭐ Meilleur Winrate ({best['main_opening']})",
        showarrow=True,
        arrowhead=2,
        ax=0,
        ay=-50,
        font=dict(color="black", size=12),
        bgcolor="lightyellow",
        bordercolor="black",
        borderwidth=1
    )

    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            direction="right",
            x=0.57, y=1.15,
            buttons=list([
                dict(label="Winrate",
                     method="update",
                     args=[
                         {"visible": [True, False]},
                         {
                             "annotations": [annotation],
                             "title": f"{side_icon} <b>Top 10 Ouvertures {side_name.title()}</b> (par Winrate)"
                         }
                     ]),
                dict(label="Nombre de parties",
                     method="update",
                     args=[
                         {"visible": [False, True]},
                         {
                             "annotations": [],
                             "title": f"{side_icon} <b>Top 10 Ouvertures {side_name.title()}</b> (par Nombre de parties)"
                         }
                     ])
            ])
        )],
        annotations=[annotation],  # Par défaut visible pour Winrate
        title=f"{side_icon} <b>Top 10 Ouvertures {side_name.title()}</b> (par Winrate)",
        xaxis_title="Ouverture",
        yaxis_title="Winrate / Nombre de parties",
        xaxis_tickangle=-45,
        bargap=0.3,
        bargroupgap=0.05,
        hoverlabel=dict(
            bgcolor="white",
            font_size=14,
            font_family="Rockwell",
            bordercolor='black'
        ),
        transition=dict(duration=500, easing='cubic-in-out'),
        plot_bgcolor='rgba(245, 245, 245, 1)',
        paper_bgcolor='rgba(245, 245, 245, 1)'
    )

    return fig

# Création des figures
fig_white = create_figure(
    top_white_winrate,
    top_white_games,
    ["#489fde", "#1f68ab", "#154d8c", "#042749", "#001029"],
    ["#cce5ff", "#3399ff", "#0055cc", "#003d99", "#001f4d"],
    "♙",
    "white"
)

fig_black = create_figure(
    top_black_winrate,
    top_black_games,
    ["#e85151", "#C22E2E", "#a30e0e", "#720000", "#4d0a0a"],
    ["#ffd6d6", "#ff9999", "#cc3333", "#800000", "#4d0000"],
    "♟️",
    "black"
)

# ----------- Sauvegarde HTML ----------- #
os.makedirs("data/exported", exist_ok=True)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
fig_white.write_html(f"data/exported/top_openings_white_{timestamp}.html", include_plotlyjs='cdn')
fig_black.write_html(f"data/exported/top_openings_black_{timestamp}.html", include_plotlyjs='cdn')

# ----------- Affichage ----------- #
fig_white.show()
fig_black.show()
